https://www.geeksforgeeks.org/movie-recommender-based-on-plot-summary-using-tf-idf-vectorization-and-cosine-similarity/?ref=rp

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
movies = pd.read_csv('/content/drive/MyDrive/CNN Practice Datasets/Movie Recommendation System/Based on movie plots/wiki_movie_plots_deduped.csv')
movies.head(3)

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."


In [3]:
movies.shape

(34886, 8)

In [4]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34886 entries, 0 to 34885
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Release Year      34886 non-null  int64 
 1   Title             34886 non-null  object
 2   Origin/Ethnicity  34886 non-null  object
 3   Director          34886 non-null  object
 4   Cast              33464 non-null  object
 5   Genre             34886 non-null  object
 6   Wiki Page         34886 non-null  object
 7   Plot              34886 non-null  object
dtypes: int64(1), object(7)
memory usage: 2.1+ MB


In [5]:
movies['Origin/Ethnicity'].unique()

array(['American', 'Australian', 'Bangladeshi', 'British', 'Canadian',
       'Chinese', 'Egyptian', 'Hong Kong', 'Filipino', 'Assamese',
       'Bengali', 'Bollywood', 'Kannada', 'Malayalam', 'Marathi',
       'Punjabi', 'Tamil', 'Telugu', 'Japanese', 'Malaysian', 'Maldivian',
       'Russian', 'South_Korean', 'Turkish'], dtype=object)

In [13]:
print(len(movies.loc[movies['Origin/Ethnicity']=='British']))
print(len(movies.loc[movies['Origin/Ethnicity']=='Bollywood']))

3670
2931


In [6]:
origin1 = pd.DataFrame(movies.loc[movies['Origin/Ethnicity']=='British'])
origin2 = pd.DataFrame(movies.loc[movies['Origin/Ethnicity']=='Bollywood'])
temp = pd.concat([origin1,origin2],ignore_index=True)

In [7]:
final_movies = temp[['Title','Plot']]
final_movies.set_index('Title')
final_movies.head()

,Title,Plot
0,The Amateur Gentleman,In Regency Britain a young man tries to establ...
1,Anna the Adventuress,Two identical sisters are able to switch place...
2,At the Villa Rose,Inspector Hanaud is asked to investigate a mur...
3,Darby and Joan,Mrs. Gorry and her son lived by an old mill st...
4,General Post,During the First World War a small-town tailor...


In [8]:
final_movies.shape

(6601, 2)

In [14]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [15]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
stpwrds = set(stopwords.words('english'))
lem =WordNetLemmatizer()
verb_codes = {'VB','VBD','VBG','VBN','VBP','VBZ'}

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [16]:
def data_preprocess(text):
  text = text.lower() #convert to lowercase
  temp = []
  tk_words = nltk.word_tokenize(text) #tokenize statements
  tags = nltk.pos_tag(tk_words) #add Parts of Speech tags to each token
  for i, word in enumerate(tk_words):
    if tags[i][1] in verb_codes:
      lemmatized = lem.lemmatize(word,'v')
    else:
      lemmatized = lem.lemmatize(word)
    if lemmatized not in stpwrds and lemmatized.isalpha():
      temp.append(lemmatized)

  result = ' '.join(temp)
  result = result.replace("n't"," not")
  result = result.replace("'m"," am")
  result = result.replace("'s"," is")
  result = result.replace("'re"," are")
  result = result.replace("'ll"," will")
  result = result.replace("'ve"," have")
  result = result.replace("'d"," would")
  return result

In [20]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [22]:
final_movies['processed_plot'] = final_movies['Plot'].apply(data_preprocess)
final_movies.head(3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Title,Plot,processed_plot
0,The Amateur Gentleman,In Regency Britain a young man tries to establ...,regency britain young man try establish father...
1,Anna the Adventuress,Two identical sisters are able to switch place...,two identical sister able switch place lead se...
2,At the Villa Rose,Inspector Hanaud is asked to investigate a mur...,inspector hanaud ask investigate murder young ...


In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [39]:
vec = TfidfVectorizer()
vec_movies = vec.fit_transform(final_movies['processed_plot'])

In [ ]:
vec = TfidfVectorizer(max_features=2500,max_df=0.5,min_df=5,stop_words=stopwords.words('english'))
vec_movies = vec.fit_transform((final_movies['processed_plot']))

In [40]:
from sklearn.metrics.pairwise import cosine_similarity

In [41]:
cos_sim = cosine_similarity(vec_movies,vec_movies)
cos_sim

array([[1.        , 0.        , 0.01213769, ..., 0.0063705 , 0.        ,
        0.00364459],
       [0.        , 1.        , 0.03062233, ..., 0.01743286, 0.00879251,
        0.01194224],
       [0.01213769, 0.03062233, 1.        , ..., 0.00842148, 0.01212541,
        0.00453134],
       ...,
       [0.0063705 , 0.01743286, 0.00842148, ..., 1.        , 0.02064546,
        0.07995828],
       [0.        , 0.00879251, 0.01212541, ..., 0.02064546, 1.        ,
        0.01558762],
       [0.00364459, 0.01194224, 0.00453134, ..., 0.07995828, 0.01558762,
        1.        ]])

In [42]:
indices = pd.Series(final_movies.index)
indices

0          0
1          1
2          2
3          3
4          4
        ... 
6596    6596
6597    6597
6598    6598
6599    6599
6600    6600
Length: 6601, dtype: int64

In [43]:
final_movies[final_movies['Title'] =="Harry Potter and the Philosopher's Stone"]

,Title,Plot,processed_plot
2741,Harry Potter and the Philosopher's Stone,"Albus Dumbledore, Minerva McGonagall, and Rube...",albus dumbledore minerva mcgonagall rubeus hag...


In [44]:
condition = final_movies['Title'] =="Harry Potter and the Philosopher's Stone"
indices[condition]

2741    2741
dtype: int64

In [45]:
indices[condition].index[0]

2741

In [46]:
cos_sim[2741]

array([0.00368379, 0.02222242, 0.00382772, ..., 0.01714246, 0.06463228,
       0.01002368])

In [47]:
type(cos_sim[2741])

numpy.ndarray

In [48]:
temp = pd.Series(cos_sim[2741])
temp

0       0.003684
1       0.022222
2       0.003828
3       0.015824
4       0.016935
          ...   
6596    0.000000
6597    0.004115
6598    0.017142
6599    0.064632
6600    0.010024
Length: 6601, dtype: float64

In [49]:
temp.sort_values(ascending=False)

2741    1.000000
3100    0.638868
3033    0.593380
2683    0.484036
1588    0.386239
          ...   
230     0.000000
1409    0.000000
232     0.000000
4799    0.000000
5673    0.000000
Length: 6601, dtype: float64

In [50]:
final_movies.index[3100]

3100

In [51]:
final_movies.iloc[3100,0]

'Harry Potter and the Deathly Hallows: Part II'

In [52]:
def recommendation(title,cosine_sim=cos_sim):
  recommended = []
  index = indices[final_movies['Title']==title].index[0]
  score = pd.Series(cosine_sim[index]).sort_values(ascending = False)
  top_10_movies = list(score.iloc[1:11].index)
  for i in top_10_movies:
    recommended.append(final_movies.iloc[i,0])
  return recommended

In [53]:
recommendation("Harry Potter and the Philosopher's Stone")

['Harry Potter and the Deathly Hallows: Part II',
 'Harry Potter and the Deathly Hallows: Part I',
 'Parting Shots',
 'Mix Me a Person',
 'Dark Blood',
 'Spooks: The Greater Good',
 'Harry Brown',
 'The Wild and the Willing',
 'Jab Harry Met Sejal',
 'Hello-Goodbye']

In [38]:
recommendation("Aashiqui")

['Jungle',
 'Dekha Pyar Tumhara',
 'Anokha Bandhan',
 'Chennai Express',
 'Ghazab Tamasha',
 'Aap Ki Khatir',
 'Aashayein',
 'Yes Boss',
 'Kuch Kuch Hota Hai',
 'Kya Yehi Pyaar Hai']